In [66]:
import pandas as ps
import numpy as np
import random
import math

In [160]:
class Config():
    T = 10 # 1000  # time
    N = 10 # 100   # number of banks
    S = 10    # number of shocks to provoke
    α = 0.3   # alpha, liquidation cost of collaterals
    φ = 0.1   # phi    attractiveness
    δ = 0.015 # delta  screening cost
    σ = 1.5   # sigma 
    depositRate = 0.03
    riskfree = 0.01
    RANDOM_CONNECTIVITY = 0.75
    TOT_SIMULATIONS = 4
    τ = 1     # tau
    η = 0.09  #  eta 
    prud = 0.045
    u = 1.0   #  ypsilon 
    
    
    iT_inv = 500
    T_inv = float(iT_inv)

    # banks parameters
    initial_loan = 120.0
    initial_liquidity = 30.0
    initial_deposit = 130.0
    initial_equity = initial_liquidity / 2
    initial_probfail = 0.2
    initial_price = 0.3
    initial_haircut = 0.3
    minInterestInterbank = 0.02
    
class Status():
    maxEquity = 0.0
    banks = []
    matched = []
    credit = []
    interestInterbank = []
    marketBank = []
    totFailures = 0
    meanRate = []
    totbaddebt = 0
    
    totloan = totliquidity = totdeposit = 0.0
    totequity = totnewdeposit = totasset = 0.0
    
    mark = 0
    
    def init():
        Status.maxEquity = 0.0
        Status.banks = []
        Status.meanRate = np.zeros( (Config.T) )
        Status.price = np.zeros( (Config.T) )
        Status.matched = np.zeros( (Config.N,Config.N) )
        Status.interestInterbank = np.zeros( (Config.N,Config.N) )  
        Status.credit = np.zeros( (Config.N,Config.N) )  
        Status.marketBank = np.zeros( (Config.N,Config.N) )  
        for i in range(Config.N):
            Status.banks.append( Bank() )
        

In [171]:
class Bank():
    def setCapacity(self):
        self.capacity = 1.0 - self.haircut*self.asset
        if self.capacity<1.0:
            self.capacity = 4.0;
        # fprintf( out49 ) **
        
    def determineProbFail(self):
        self.probfail = 1.0 - self.equity / Status.maxEquity
            
    def __init__(self):
        self.loan = Config.initial_loan
        self.liquidity = Config.initial_liquidity
        self.deposit = Config.initial_deposit
        self.equity = Config.initial_equity
        self.probfail = Config.initial_probfail
        self.haircut = Config.initial_haircut
        self.newDeposit = self.dLoan  = 0.0
        self.interbankLoan = 0.0
        self.interbankDebt = 0.0
        self.failB = 0.0
        self.asset = 0.0
        self.rationed = 0
        self.badDebt = 0.0
        self.leverage = 0.0
        self.deltaD = self.firesale = self.crunch =  0.0
        self.loanintero = 0.0
        self.asset = self.loan+ self.liquidity
        self.leverage = self.loan / self.equity
        
        self.rate = 0.0       # tasso
        self.totalRate = 0.0  # sommatasso
        self.connected = 0.0  # collegati
        
        self.marked = 0
        self.neighbour = 0   
        
        self.a = 0.0
        self.b = 0.0
        self.c = 0.0
        
        self.setCapacity()


In [173]:
def matrixRandom():
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 0.0
        for j in range(len(Status.banks)):
            Status.marketBank[i][j] = 0.0

    for i in range(len(Status.banks)):
        if Status.banks[i].connected != 1:
            aux = random.random()
            if aux < Config.RANDOM_CONNECTIVITY:
                j=i
                while i==j:
                    j = int(random.random() * Config.N)
                    if i!=j:
                        Status.marketBank[i][j]=1
                        Status.banks[i].neighbour = j
                        Status.banks[i].connected = 1

    for i in range(len(Status.banks)):
        Status.banks[i].marked = -1
    for i in range(len(Status.banks)):
        if Status.banks[i].marked == -1:
            Status.mark += 1
            matrixRandom_dfs(i)
    
                        
def matrixRandom_dfs(n):
    if (Status.banks[n].marked == -1):
        Status.banks[n].marked = Status.mark
        for i in range(len(Status.banks)):
            if (Status.marketBank[n][i] == 1 or Status.marketBank[i][n] == 1) and Status.banks[i].marked == -1 :
                matrixRandom_dfs(i)

def matrixPreferential():
    k = 0
    switching = 0
    ref = 0.0
    fact = 0.0
    prob = 0.0
    fitness = 0.0
    liquidityMax = -1000.0
    interestMin = 1000.0
    i_max = -1
    i_max2= -1
    for j in range(len(Status.banks)):
        if Status.banks[j].liquidity > liquidityMax:
            liquidityMax = Status.banks[j].liquidity
            i_max = j
        if Status.banks[j].rate < interestMin:
            interestMin = Status.banks[j].rate
            i_max2= j

    β = np.zeros( (Config.N) )
    for i in range(len(Status.banks)):
        Status.banks[i].fit = 0.0
        Status.banks[i].outgoingLink = 0
        Status.banks[i].incomingLink = 0
        Status.banks[i].link = 0
    for j in range(len(Status.banks)):
        Status.banks[j].fit = \
            Config.u * Status.banks[j].liquidity / liquidityMax + (1-Config.u)* interestMin * Status.banks[j].rate 
        # fprintf( out20 ) **

    Status.marketBank = np.zeros( (Config.N,Config.N), int )
    for i in range(len(Status.banks)):
        β[i] = Config.T_inv
    for i in range(len(Status.banks)):
        j=i
        while j==i:
            j=int(random.random()*Config.N)
        k = Status.banks[i].neighbour
        prob = random.random()
        fitness = Status.banks[j].fit - Status.banks[k].fit 
        ref = β[i] * fitness
        fact = 1.0 / (1.0 + math.exp(-ref))
        if (prob<fact) and (i!=j):
            switching += 1
            Status.marketBank[i][k] = 0
            Status.marketBank[i][j] = 1
            Status.banks[i].neighbour = j
        else:
            Status.marketBank[i][j] = 0
            Status.marketBank[i][k] = 1
            Status.banks[i].neighbour = k
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 1
    
    
    
def findMaxEquity():
    Status.maxEquity = 0.0
    for i in range(len(Status.banks)):
        if (Status.banks[i].equity>Status.maxEquity):
            Status.maxEquity = Status.banks[i].equity
            
def reviewBanksInterbankRate():
    for j in range(len(Status.banks)):
        for i in range(len(Status.banks)):
            if not Status.credit[j][i]:
                Status.banks[j].a = Config.prud * Status.banks[i].asset   +  Config.δ *  Status.banks[i].asset 
                
                Status.banks[j].b = Status.banks[i].probfail * (Config.α*Status.banks[i].asset -  Status.banks[i].capacity)
                
                Status.banks[j].c = ( 1 - Status.banks[i].probfail ) * Status.banks[i].capacity 
                
                Status.interestInterbank[j][i] = ( Status.banks[j].a - Status.banks[j].b ) / Status.banks[j].c
                if (Status.interestInterbank[j][i]< Config.minInterestInterbank):
                    Status.interestInterbank[j][i] = Config.minInterestInterbank

def reviewBanksRate():
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 0.0
        Status.banks[i].totalRate = 0.0
        
    for j in range(len(Status.banks)):
        for i in range(len(Status.banks)):
            if not (Status.credit[j][i]):                
                Status.banks[j].connected += 1 
                Status.banks[j].totalRate += Status.interestInterbank[j][i]
                Status.banks[j].rate = Status.banks[j].totalRate / Status.banks[j].connected

def getGurus(numGurus):
    gurus = []
    for i in range(numGurus):
        incominglinkMax = -1000.0
        guru = -1
        for j in range(len(Status.banks)):
            if j not in gurus and Status.banks[j].incomingLink > incominglinkMax:
                incominglinkMax = Status.banks[j].incomingLink
                gurus.append( j )
    return gurus

def getSumIncomingLinksGurus( gurus ):
    total = 0
    for i in gurus:
        total += Status.banks[i].incomingLink
    return total

def updateMortigurus( i ):
    for k in range(len(Status.gurus)):
        if Status.banks[i].neighbour == Status.gurus[k]:
            Status.mortigurus[k] += 1
            
def newBanks():
    for i in range(len(Status.banks)): 
        if Status.banks[i].equity <= 0 or Status.banks[i].loan < 0:
            Status.banks[i].failB = 1
    for i in range(len(Status.banks)): 
        for j in range(len(Status.banks)): 
            if Status.banks[i].failB == 1 or Status.banks[j].failB == 1:
                Status.credit[j][i] = 0.0
    for i in range(len(Status.banks)): 
        if Status.banks[i].failB == 1:
            Status.banks[i] = Bank()

            
def firesale(t):
    for i in range(len(Status.banks)):    
        if Status.banks[i].dLoan > 0.0:
            Status.banks[i].rationed = 1
            Status.banks[i].firesale = Status.banks[i].dLoan / Status.price[t]
            Status.banks[i].loanintero = Status.banks[i].loan
            Status.banks[i].loan -= Status.banks[i].firesale
            if Status.banks[i].loan >= 0.0:
                Status.banks[i].dLoan = 0.0
                Status.banks[i].crunch = Status.banks[i].dLoan
                Status.banks[i].equity -= (1 - Status.price[t]) * Status.banks[i].firesale
                if Status.banks[i].equity <= 0.0:
                    Status.banks[i].interbankDebt = 0.0
                    Status.banks[i].failB = 1
                    Status.banks[i].executed = 0
            else:
                Status.banks[i].dLoanintera = Status.banks[i].dLoan;
                Status.banks[i].dLoan -= Status.price[t] * Status.banks[i].loanintero
                Status.banks[i].crunch = Status.banks[i].dLoan
                Status.banks[i].dLoan = 0.0
                Status.banks[i].interbankDebt = 0.0
                Status.banks[i].failB = 1
                Status.banks[i].executed = 0
    totrationed = 0
    for i in range(len(Status.banks)):    
        totrationed += Status.banks[i].rationed
    # fprintf( out43 ) **
  
    totrichesto = 0.0
    totconcesso = 0
    for i in range(len(Status.banks)):    
        if Status.banks[i].rationed == 1:
            totrichesto += Status.banks[i].richiesta
            totconcesso += Status.banks[i].concesso
    # fprintf( out44 ) **

    meanrationcore = meanrationperi = totrationcore = totrationperi = 0.0
    for i in range(len(Status.banks)):    
        if Status.banks[i].rationed == 1:
            if Status.banks[i].core == 1:
                totrationcore += 1
            else:
                totrationperi += 1
                
    meanrationcore = totrationcore / coretot
    meanrationperi = totrationperi / peripherytot
    # fprintf( out73 ) **
    # fprintf( out74 ) **
    
    for i in range(len(Status.banks)):    
        Status.banks[i].rationed == 0.0
    for i in range(len(Status.banks)):    
        for j in range(len(Status.banks)):    
            if Status.banks[i].failB == 1:
                Status.banks[i].loan = 0.0
                Status.matched[i][j] = 0
                Status.banks[j].equity -= Status.credit[j][i]
                Status.banks[j].interbankLoan -= Status.credit[j][i]
                Status.banks[j].badDebt += (Status.credit[j][i] * (1. + Status.interestInterbank[j][i]))
                Status.banks[i].interbankDebt = 0.0
                Status.credit[j][i] = 0.0
                Status.banks[i].failB = 1
                Status.banks[j].executed = 0

    
def aggregateStatistics():
    Status.totloan = Status.totliquidity = Status.totdeposit = 0.0
    Status.totequity = Status.totnewdeposit = Status.totasset = 0.0
    for i in range(len(Status.banks)):
        Status.totloan = Status.banks[i].loan
        Status.totliquidity +=  Status.banks[i].liquidity
        Status.totdeposit += Status.banks[i].deposit
        Status.totequity +=  Status.banks[i].equity
        Status.totnewdeposit +=  Status.banks[i].newDeposit
        Status.totasset +=  Status.banks[i].asset
        # fprintf( out14 ) **
        # fprintf( out15 ) **
        # fprintf( out16 ) **
        # fprintf( out17 ) **
        # fprintf( out45 ) **

def trade():
    for i in range(len(Status.banks)):
        Status.banks[i].insideinter = 0
    for i in range(len(Status.banks)):
        for j in range(len(Status.banks)):
            if Status.banks[i].dLoan > 0.0 and Status.marketBank[i][j] == 1 and Status.banks[j].liquidity > 0.0:
                Status.banks[j].insideinter = 1
                Status.matched[i][j] = 1
                if Status.banks[i].dLoan < Status.banks[j].liquidity:
                    Status.banks[j].liquidity -= Status.banks[i].dLoan
                    Status.banks[j].interbankLoan += Status.banks[i].dLoan
                    Status.banks[i].interbankDebt = Status.banks[i].dLoan
                    Status.credit[j][i] =Status.banks[i].dLoan
                    Status.banks[i].concesso = Status.banks[i].dLoan
                    Status.banks[i].dLoan = 0.0
                else:
                    Status.banks[j].interbankLoan += Status.banks[j].liquidity
                    Status.banks[i].interbankDebt = Status.banks[j].liquidity
                    Status.credit[j][i] = Status.banks[j].liquidity
                    Status.banks[i].concesso = Status.banks[j].liquidity
                    Status.banks[i].dLoan -= Status.banks[j].liquidity
                    Status.banks[j].liquidity = 0.0
                    
    for i in range(len(Status.banks)):
        for j in range(len(Status.banks)):
            if Status.marketBank[i][j] == 1 and Status.credit[j][i] != 0.0:
                # fprintf( out58 ) **
                pass
    # fprintf( out11 ) **
  

In [175]:
for simulation in range(Config.TOT_SIMULATIONS):
    Status.init()
    for t in range(Config.T):
        if (t<10):
            matrixRandom()
        else:
            
            for i in range(len(Status.banks)):
                Status.banks[i].setCapacity()
                findMaxEquity()
                Status.banks[i].determineProbFail()
                
            reviewBanksInterbankRate()
            reviewBanksRate()
            
            matrixPreferential()
        
        # fprintf( out57 ) **
       
        Status.totFailures = 0
        for i in range(len(Status.banks)):
            Status.banks[i].incomingLink = 0
            
        for i in range(len(Status.banks)):
            for j in range(len(Status.banks)):
                if Status.marketBank[i][j]==1 and i!=j:
                    Status.banks[j].incomingLink += 1
        # fprintf( out12 ) **

        probfaillinked = probfailnonlinked = 0.0
        num = num1 = guru = 0
        for i in range(len(Status.banks)):
            if Status.banks[j].neighbour == guru:
                probfaillinked += Status.banks[i].probfail
                num += 1
                probfaillinked += probfaillinked / num
            else:
                probfailnonlinked += Status.banks[i].probfail
                num1+= 1
                probfailnonlinked += probfailnonlinked / num1
                
        Status.gurus = getGurus(10)
        
        for i in range(len(Status.banks)):
            Status.banks[i].core = 0
            Status.banks[i].periphery = 0
        coretot = peripherytot = 0
        inlinkcore = inlinkperi = 0
        
        threshold = 0.5 * ( getSumIncomingLinksGurus(Status.gurus) / 10 )
        for i in range(len(Status.banks)):
            if (Status.banks[i].incomingLink>=threshold):
                Status.banks[i].core = 1
                coretot += 1
                inlinkcore += Status.banks[i].incomingLink
            else:
                Status.banks[i].periphery = 1
                peripherytot += 1
                inlinkperi += Status.banks[i].incomingLink
        meanlinkcore = inlinkcore / coretot
        meanlinkperi = inlinkperi / peripherytot
        
        # fprintf( out60 ) **
        # fprintf( out62 ) **
        # fprintf( out63 ) **        

        Status.meanInterestCore = np.zeros( ( Config.N ) )
        Status.meanInterestPeri = np.zeros( ( Config.N ) )
        for j in range(len(Status.banks)):
            if Status.banks[j].core == 1:
                for i in range(len(Status.banks)):
                    Status.meanInterestCore[j] += Status.interestInterbank[j][i]
            else:
                for i in range(len(Status.banks)):
                    Status.meanInterestPeri[j] += Status.interestInterbank[j][i]
        for j in range(len(Status.banks)):
            if Status.banks[j].core == 1:
                Status.meanInterestCore[j] /= 100
            else:
                Status.meanInterestPeri[j] /= 100

        meanintcore = meanintperi = 0.0
        for j in range(len(Status.banks)):
            if Status.banks[j].core == 1:
                meanintcore += Status.meanInterestCore[j]
            else:
                meanintperi += Status.meanInterestPeri[j]
        meanintcore /= coretot
        meanintperi /= peripherytot
        
        # fprintf( out64 ) **        

        meancapacitycore = meancapacityperi = totcapacitycore = totcapacityperi = 0.0
        for i in range(len(Status.banks)):
            if Status.banks[i].core == 1:
                totcapacitycore += Status.banks[i].core
            else:
                totcapacityperi += Status.banks[i].core 
        meancapacitycore /= coretot
        meancapacityperi /= peripherytot
        
        # fprintf( out71 ) **        
        # fprintf( out72 ) **        

        guruperc = 0.0
        incominglinkperc = 0.0
        enne = 0.0
        minchia = 0.0
        minchia1 = 0.0
        enne = Config.N
        minchia = Status.gurus[0]
        minchia1 = Status.banks[ Status.gurus[0] ].incomingLink
        guruperc = (minchia / enne)
        incominglinkperc = (minchia1 / enne)
        print("uno")

        # fprintf( out8 ) **  
        # fprintf( out21 ) **    
        # fprintf( out1 ) **          
        # fprintf( out2 ) **

        # fprintf( out24 ) **  
        # fprintf( out25 ) ** 

        depositimedi = 0.0
        equitymedia = 0.0
        baddebtmedi = 0.0
        liquiditymedia = 0.0
        assetmedio = 0.0

        depositimedi = Status.totdeposit - Status.banks[Status.gurus[0]].deposit
        depositimedi = depositimedi / (Config.N - 1)

        equitymedia = Status.totequity - Status.banks[Status.gurus[0]].equity
        equitymedia = equitymedia / (Config.N - 1)

        baddebtmedi = Status.totbaddebt - Status.banks[Status.gurus[0]].badDebt
        baddebtmedi = baddebtmedi / (Config.N - 1)

        liquiditymedia = Status.totliquidity - Status.banks[Status.gurus[0]].liquidity
        liquiditymedia = liquiditymedia / (Config.N - 1)

        assetmedio = Status.totasset - Status.banks[Status.gurus[0]].asset
        assetmedio = assetmedio / (Config.N - 1)

        # fprintf( out ) ** 
        # fprintf( out19 ) ** 
        # fprintf( out27) ** 
        # fprintf( out29) ** 
        # fprintf( out46 ) ** 


        print("tres")
        Status.totbaddebt = 0.0
        Status.price[t] = 0.001
        
        for i in range(len(Status.banks)):
            Status.banks[i].richiesta = 0.0
            Status.banks[i].concesso = 0.0
            Status.banks[i].shocked = 0
            
        for z in range(Config.S):
            i = int( random.random()*Config.N )
            if Status.banks[i].shocked != 1:
                Status.banks[i].shocked = 1
            else:
                while (Status.banks[i].shocked==1):
                    i = int( random.random()*Config.N )
                Status.banks[i].shocked = 1
       
        for i in range(len(Status.banks)):
            if Status.banks[i].shocked==1:
                Status.banks[i].newDeposit = ((random.random() * 0.6) + 0.65) * Status.banks[i].deposit
            else:
                Status.banks[i].newDeposit = Status.banks[i].deposit
        
        
            Status.banks[i].deltaD = Status.banks[i].newDeposit - Status.banks[i].deposit
            Status.banks[i].liquidity += Status.banks[i].deltaD
            Status.banks[i].deposit = Status.banks[i].newDeposit
            Status.banks[i].asset = Status.banks[i].loan + Status.banks[i].liquidity
            Status.banks[i].leverage = Status.banks[i].loan / Status.banks[i].equity
            
            if Status.banks[i].liquidity<0:
                Status.banks[i].dLoan = math.fabs(Status.banks[i].liquidity)
                Status.banks[i].liquidity = 0.0
            else:
                Status.banks[i].dLoan = 0.0
        
        print("cuatro")
        # fprintf( out27) ** 
        # fprintf( out29) ** 
        # fprintf( out46 ) **  
        asked = 0.0
        for i in range(len(Status.banks)):
            asked += Status.banks[i].dLoan
            Status.banks[i].newDeposit = 0.0
        # fprintf( out4 ) **  

        
        Status.mortigurus = np.zeros( (len(Status.gurus)), int )
        meanmorticore = meanmortiperi = totmorticore = totmortiperi = 0.0
        for i in range(len(Status.banks)):
            if Status.banks[i].failB == 1:
                totmorticore += 1
            else:
                totmortiperi += 1
        meanmorticore = totmorticore / coretot
        meanmortiperi = totmortiperi / peripherytot
        # fprintf( out65 ) **  
        # fprintf( out66 ) **  
        
        for i in range(len(Status.banks)):
            if Status.banks[i].failB == 1:
                updateMortigurus(i)
        meanbaddebtcore = meanbaddebtperi = totbaddebtcore = totbaddebtperi = 0.0
        for i in range(len(Status.banks)):
            if Status.banks[i].core == 1:
                totbaddebtcore += Status.banks[i].badDebt
            else:
                totbaddebtperi += Status.banks[i].badDebt
            
        meanbaddebtcore = totbaddebtcore / coretot
        meanbaddebtperi = totbaddebtperi / peripherytot
        # fprintf( out67 ) **  
        # fprintf( out68 ) **  

        
        print("cinco")
        
        totbaddebt = 0.0
        for i in range(len(Status.banks)):
            totbaddebt += Status.banks[i].badDebt
        # fprintf( out18 ) **
        
        newBanks()
        for i in range(len(Status.banks)):
            Status.banks[i].badDebt = 0.0
            Status.banks[i].executed = 0

        crunchTot = 0.0
        for i in range(len(Status.banks)):
            if Status.banks[i].dLoan > 0 and Status.banks[i].connected == 0:
                Status.banks[i].firesale = Status.banks[i].dLoan / Status.price[t]
                Status.banks[i].loanIntero =Status.banks[i].loan
                Status.banks[i].loan -= Status.banks[i].firesale
                if Status.banks[i].loan>=0:
                    Status.banks[i].dLoan = 0.0
                    Status.banks[i].crunch = Status.banks[i].dLoan
                    Status.banks[i].equity -= (1 -  Status.price[t] ) * Status.banks[i].firesale
                    Status.banks[i].newDeposit = 0.0
                    Status.banks[i].deltaD = 0.0
                    Status.banks[i].firesale = 0.0
                    Status.banks[i].interbankLoan = 0.0
                    Status.banks[i].interbankDebt = 0.0
                    Status.banks[i].failB = 0
                    Status.banks[i].asset = Status.banks[i].loan + Status.banks[i].liquidity
                    Status.banks[i].leverage = Status.banks[i].loan / Status.banks[i].equity
                else:
                    
                    Status.banks[i].dLoan -= Status.price[t] * Status.banks[i].loanintero
                    Status.banks[i].crunch = Status.banks[i].dLoan
                    Status.banks[i].failB = 1
                    Status.banks[i].dLoan = 0.0
                    Status.banks[i].equity = 0.0

                crunchTot += Status.banks[i].crunch
        print("seis")
      
        trade()
        for i in range(len(Status.banks)):
            Status.banks[i].intradayLeverage = 0.0
            
        for i in range(len(Status.banks)):
            Status.banks[i].intradayLeverage = ( Status.banks[i].loan + Status.banks[i].interbankLoan ) \
                                                 / 100 # Status.banks[i].equity
        # fprintf( out59 ) ** 

        meanlevacore = meanlevaperi = totlevacore = totlevaperi = 0.0
        for i in range(len(Status.banks)):
            if Status.banks[i].core == 1:
                totlevacore += Status.banks[i].intradayLeverage
            else:
                totlevaperi += Status.banks[i].intradayLeverage

        meanlevacore = totlevacore / coretot
        meanlevaperi = totlevaperi / peripherytot
        # fprintf( out69 ) ** 
        # fprintf( out70 ) ** 

        print("siete")

        granted = 0.0
        for i in range(len(Status.banks)):
            for j in range(len(Status.banks)):
                granted += Status.credit[j][i]
        # fprintf( out5) ** 

        
        loanToRate = transaction = 0.0
        Status.meanRate[t] = 0.0
        Status.meanRate[1] = 0.04
        for i in range(len(Status.banks)):
            for j in range(len(Status.banks)):
                loanToRate += Status.credit[j][i] * Status.interestInterbank[j][i]
                transaction += Status.credit[j][i]
                
        if transaction != 0.0:
            Status.meanRate[t] = loanToRate / transaction
        else:
            Status.meanRate[t] = Status.meanRate[t-1]
        # fprintf( out6 ) ** 

        firesale(t)
        for i in range(len(Status.banks)):
            Status.totFailures += Status.banks[i].failB  
        # fprintf( out7 ) ** 

        
        no_connect_fail = 0
        for i in range(len(Status.banks)):
            if Status.banks[i].connected == 0:
                no_connect_fail += Status.banks[i].failB  
        # fprintf( out26 ) ** 


        print("ocho")
        newBanks()
        aggregateStatistics()


        maxleverage = 0.0
        for i in range(len(Status.banks)):
            for j in range(len(Status.banks)):
                if Status.matched[i][j] == 0:
                    if Status.banks[i].leverage > maxleverage:
                        maxleverage = Status.banks[i].leverage 
        for i in range(len(Status.banks)):
            Status.banks[i].haircut =  Status.banks[i].leverage / maxleverage
        

                    
      

uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
siete
ocho
uno
uno
tres
cuatro
cinco
seis
sie

In [136]:
Status.banks